In [89]:
import os
import sys
import math
import keras
import datetime
import itertools
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [2]:
# Append TSPerf path to sys.path
nb_dir = os.path.split(os.getcwd())[0]
tsperf_dir = os.path.dirname(os.path.dirname(os.path.dirname(nb_dir)))
if tsperf_dir not in sys.path:
    sys.path.append(tsperf_dir)

from common.metrics import MAPE
import retail_sales.OrangeJuice_Pt_3Weeks_Weekly.common.benchmark_settings as bs

In [63]:
# Data paths
DATA_DIR = '../../data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

# Parameters of the model
PRED_HORIZON = 3
PRED_STEPS = 2
SEQ_LEN = 8
DYNAMIC_FEATURES = ['deal', 'feat']
STATIC_FEATURES = ['store', 'brand']

In [106]:
def df_from_cartesian_product(dict_in):
    """Generate a Pandas dataframe from Cartesian product of lists.
    
    Args: 
        dict_in (Dictionary): Dictionary containing multiple lists
        
    Returns:
        df (Dataframe): Dataframe corresponding to the Caresian product of the lists
    """
    from collections import OrderedDict
    from itertools import product
    od = OrderedDict(sorted(dict_in.items()))
    cart = list(product(*od.values()))
    df = pd.DataFrame(cart, columns=od.keys())
    return df

def gen_sequence(df, seq_len, seq_cols, start_timestep=0, end_timestep=None):
    """Reshape features into an array of dimension (time steps, features).  
    
    Args:
        df (Dataframe): Time series data of a specific (store, brand) combination
        seq_len (Integer): The number of previous time series values to use as input features
        seq_cols (List): A list of names of the feature columns 
        start_timestep (Integer): First time step you can use to create feature sequences
        end_timestep (Integer): Last time step you can use to create feature sequences
        
    Returns:
        A generator object for iterating all the feature sequences
    """
    data_array = df[seq_cols].values
    if end_timestep is None:
        end_timestep = df.shape[0]
    for start, stop in zip(range(start_timestep, end_timestep-seq_len+1), range(start_timestep+seq_len, end_timestep+1)):
        yield data_array[start:stop, :]


def gen_sequence_array(df_all, seq_len, seq_cols, start_timestep=0, end_timestep=None):
    """Combine feature sequences for all the combinations of (store, brand) into an 3d array.
    
    Args:
        df_all (Dataframe): Time series data of all stores and brands
        seq_len (Integer): The number of previous time series values to use as input features
        seq_cols (List): A list of names of the feature columns 
        start_timestep (Integer): First time step you can use to create feature sequences
        end_timestep (Integer): Last time step you can use to create feature sequences
        
    Returns:
        seq_array (Numpy Array): An array of the feature sequences of all stores and brands    
    """
    seq_gen = (list(gen_sequence(df_all[(df_all['store']==cur_store) & (df_all['brand']==cur_brand)], \
                                 seq_len, seq_cols, start_timestep, end_timestep)) \
              for cur_store, cur_brand in itertools.product(df_all['store'].unique(), df_all['brand'].unique()))
    seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
    return seq_array

def static_feature_array(df_all, total_timesteps, seq_cols):
    """Generate an arary which encodes all the static features.
    """
    fea_df = df_all.groupby(['store', 'brand']). \
                              apply(lambda x: x.loc[:total_timesteps,:])
    fea_array = fea_df[seq_cols].values
    return fea_array

In [5]:
r = 0
print('---- Round ' + str(r+1) + ' ----')
train_df = pd.read_csv(os.path.join(TRAIN_DIR, 'train_round_'+str(r+1)+'.csv'))
train_df['move'] = train_df['logmove'].apply(lambda x: round(math.exp(x)))
train_df.drop('logmove', axis=1, inplace=True)
print(train_df.head(3))
print('')
# Fill missing values
store_list = train_df['store'].unique()
brand_list = train_df['brand'].unique()
week_list = range(bs.TRAIN_START_WEEK, bs.TEST_END_WEEK_LIST[r]+1)
d = {'store': store_list,
     'brand': brand_list,
     'week': week_list}        
data_grid = df_from_cartesian_product(d)
data_filled = pd.merge(data_grid, train_df, how='left', 
                        on=['store', 'brand', 'week'])
print('Number of missing rows is {}'.format(data_filled[data_filled.isnull().any(axis=1)].shape[0]))
print('')
data_filled = data_filled.groupby(['store', 'brand']). \
                          apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
print(data_filled.head(3))
print('')

---- Round 1 ----
   store  brand  week  constant    price1    price2    price3    price4  \
0      2      1    40         1  0.060469  0.060497  0.042031  0.029531   
1      2      1    46         1  0.060469  0.060312  0.045156  0.046719   
2      2      1    47         1  0.060469  0.060312  0.045156  0.046719   

     price5    price6    price7    price8    price9   price10   price11  deal  \
0  0.049531  0.053021  0.038906  0.041406  0.028906  0.024844  0.038984     1   
1  0.049531  0.047813  0.045781  0.027969  0.042969  0.042031  0.038984     0   
2  0.037344  0.053021  0.045781  0.041406  0.048125  0.032656  0.038984     0   

   feat     profit  move  
0   0.0  37.992326  8256  
1   0.0  30.126667  6144  
2   0.0  30.000000  3840  

Number of missing rows is 6204

   brand  store  week  constant    price1    price2    price3    price4  \
0      1      2    40       1.0  0.060469  0.060497  0.042031  0.029531   
1      1      2    41       1.0  0.060469  0.060497  0.042031  0.

In [127]:
# Normalize the dataframe of features
cols_normalize = data_filled.columns.difference(['store','brand','week'])
min_max_scaler = MinMaxScaler()
data_filled_scaled = pd.DataFrame(min_max_scaler.fit_transform(data_filled[cols_normalize]), 
                                  columns=cols_normalize, 
                                  index=data_filled.index)
data_filled_scaled.head()

,constant,deal,feat,move,price1,price10,price11,price2,price3,price4,price5,price6,price7,price8,price9,profit
0,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
1,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
2,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
3,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567
4,0.0,1.0,0.0,0.011436,1.0,0.493088,1.0,0.993939,0.686433,0.434783,1.0,1.0,0.731481,0.977528,0.485356,0.38567


In [68]:
data_sub = data_filled[(data_filled.brand==1) & (data_filled.store==2)]
data_sub.shape

(99, 19)

In [120]:
start_timestep = 0
end_timestep = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK-PRED_HORIZON+1

train_input1 = gen_sequence_array(data_filled, SEQ_LEN, ['move'], start_timestep, end_timestep)

train_input1.shape

(78518, 8, 1)

In [121]:
start_timestep = PRED_HORIZON
end_timestep = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK+1

train_input2 = gen_sequence_array(data_filled, SEQ_LEN, DYNAMIC_FEATURES, start_timestep, end_timestep)

train_input2.shape

(78518, 8, 2)

In [105]:
train_input = np.concatenate((train_input1, train_input2), axis=2)
train_input.shape

(78518, 8, 3)

In [125]:
#train_output
start_timestep = SEQ_LEN+1
end_timestep = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK+1
train_output = gen_sequence_array(data_filled, PRED_STEPS, ['move'], start_timestep, end_timestep)
train_output = np.squeeze(train_output)
train_output.shape

(78518, 2)

(78518, 2)

In [116]:
data_sub[['store','brand','week','move','deal','feat']]

,store,brand,week,move,deal,feat
0,2,1,40,8256.0,1.0,0.0
1,2,1,41,8256.0,1.0,0.0
2,2,1,42,8256.0,1.0,0.0
3,2,1,43,8256.0,1.0,0.0
4,2,1,44,8256.0,1.0,0.0
5,2,1,45,8256.0,1.0,0.0
6,2,1,46,6144.0,0.0,0.0
7,2,1,47,3840.0,0.0,0.0
8,2,1,48,8000.0,0.0,0.0
9,2,1,49,8000.0,0.0,0.0


In [43]:
store_col = np.full([train_input1.shape[0],1], 1)
brand_col = np.full([train_input1.shape[0],1], 2)
np.concatenate((store_col, brand_col), axis=1)

(88, 2)

In [49]:
bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK-sequence_len+1

88

In [14]:
# Test MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data_sub)
data_sub_scaled = scaler.transform(data_sub)
data_sub_scaled

array([[0.        , 0.        , 0.        , ..., 0.        , 0.73084502,
        0.11508554],
       [0.        , 0.        , 0.01020408, ..., 0.        , 0.73084502,
        0.11508554],
       [0.        , 0.        , 0.02040816, ..., 0.        , 0.73084502,
        0.11508554],
       ...,
       [0.        , 0.        , 0.97959184, ..., 0.        , 0.3278259 ,
        0.21617418],
       [0.        , 0.        , 0.98979592, ..., 0.        , 0.3278259 ,
        0.21617418],
       [0.        , 0.        , 1.        , ..., 0.        , 0.3278259 ,
        0.21617418]])

In [17]:
data_sub_scaled[0,]

array([0.        , 0.        , 0.        , 0.        , 1.        ,
       1.        , 0.62921348, 0.25477707, 1.        , 1.        ,
       0.70103093, 0.97435897, 0.41148325, 0.35294118, 1.        ,
       1.        , 0.        , 0.73084502, 0.11508554])

In [12]:
sa = gen_sequence_array(data_sub, sequence_length, sequence_cols)
sa.shape

(91, 8, 2)

In [85]:
total_timesteps = bs.TRAIN_END_WEEK_LIST[r]-bs.TRAIN_START_WEEK-sequence_len+1
train_input2 = static_feature_array(data_sub, total_timesteps, ['store', 'brand'])

In [60]:
np.concatenate((train_input2, train_input2)).shape

(178, 2)

In [10]:
# Model definition
n_steps = 40
n_filters = 32
from keras.models import Sequential, Model
from keras.layers import * #Conv1D, Dense
#sales_in = Input(shape=(n_steps, 1))
#c1 = Conv1D(n_filters, 2, dilation_rate=1, padding='causal', activation='relu')(sales_in)
#c1 = Flatten()(c1)
#output = Dense(2, activation='relu')(c1)
model = Sequential([
    Conv1D(n_filters, 2, dilation_rate=2, padding='causal', activation='relu', input_shape=(n_steps,1)),
    Flatten(),
    Dense(2, activation='relu')
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 32)            96        
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 2,658
Trainable params: 2,658
Non-trainable params: 0
_________________________________________________________________
